In [2]:
import pandas as pd
import json

# Load JSON file
with open("data/cvas_data.json", "r") as f:
    data = json.load(f)

# Check the top-level structure
print(type(data))       # should be dict
print(data.keys())      # should contain 'data'

<class 'dict'>
dict_keys(['data'])


In [3]:
# Extract all loans into one flat list
all_loans = []
for customer in data["data"]:
    for loan in customer["loans"]:
        all_loans.append(loan)

# Convert to DataFrame
df = pd.DataFrame(all_loans)

# Show the first few rows
df.head()

,customer_ID,loan_date,amount,fee,loan_status,term,annual_income
0,1090,15/11/2021,2426,199,0,long,41333
1,3565,07/03/2021,2153,53,0,short,76498
2,3565,06/08/2021,1538,89,1,long,76498
3,3565,30/09/2021,2997,24,0,short,76498
4,3565,09/03/2021,2184,38,0,long,76498


In [5]:
# Check columns and types
df.info()

# Preview unique terms
print(df["term"].unique())

# Check basic stats
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customer_ID    151 non-null    object
 1   loan_date      151 non-null    object
 2   amount         151 non-null    object
 3   fee            151 non-null    object
 4   loan_status    151 non-null    object
 5   term           151 non-null    object
 6   annual_income  151 non-null    object
dtypes: object(7)
memory usage: 8.4+ KB
['long' 'short']


,customer_ID,loan_date,amount,fee,loan_status,term,annual_income
count,151,151,151,151,151,151,151
unique,100,147,147,99,2,2,100
top,3565,26/05/2021,842,125,0,short,76498
freq,7,2,2,5,78,80,7


In [6]:
# Convert columns to numeric
df["amount"] = pd.to_numeric(df["amount"])
df["fee"] = pd.to_numeric(df["fee"])
df["loan_status"] = pd.to_numeric(df["loan_status"])
df["annual_income"] = pd.to_numeric(df["annual_income"])


In [13]:
features = df.groupby("customer_ID").agg({
    "amount": ["sum", "mean", "count"],
    "fee": "mean",
    "loan_status": "mean",   # This gives the default rate
    "term": lambda x: x.mode()[0] if not x.mode().empty else "unknown",
    "annual_income": "first" # Same across rows for each customer
})

In [14]:
# Flatten column names
features.columns = ['_'.join(col).strip('_') for col in features.columns]
features.reset_index(inplace=True)

# Preview
features.head()


,customer_ID,amount_sum,amount_mean,amount_count,fee_mean,loan_status_mean,term_<lambda>,annual_income_first
0,1090,2426,2426.0,1,199.0,0.0,long,41333
1,1159,2581,2581.0,1,117.0,1.0,short,46330
2,124,785,785.0,1,173.0,0.0,short,69246
3,1241,2888,2888.0,1,75.0,0.0,short,36275
4,125,2626,2626.0,1,124.0,0.0,short,77110


In [16]:
features.rename(columns={
    "amount_sum": "total_loan",
    "amount_mean": "avg_loan",
    "amount_count": "loan_count",
    "fee_mean": "avg_fee",
    "loan_status_mean": "default_rate",
    "term_<lambda_0>": "most_common_term",
    "annual_income_first": "income"
}, inplace=True)

In [17]:
# Preview
features.head()

,customer_ID,total_loan,avg_loan,loan_count,avg_fee,default_rate,term_<lambda>,income
0,1090,2426,2426.0,1,199.0,0.0,long,41333
1,1159,2581,2581.0,1,117.0,1.0,short,46330
2,124,785,785.0,1,173.0,0.0,short,69246
3,1241,2888,2888.0,1,75.0,0.0,short,36275
4,125,2626,2626.0,1,124.0,0.0,short,77110


In [19]:
features.to_csv("data/customer_features.csv", index=False)